- create a new anaconda environment to manage libs and dependancies for this project, and name it choir_network `conda create -n choir_network python=3.9`
- y when prompted
- completely close VS code and reopen this folder (recents) - a refresh is needed
- activate the environment `conda activate choir_network`
- in the upper right corner, select the kernel for the notebook `select another > python envs > click on choir_network`
- now conda install stuff. Run `conda install scikit-learn` in the terminal while you see `(choir_network)`, press `y` as needed.
- run your code. If stuff asks to install `ipykernel package` stuff, just say yes
- let's see if u can get past the matplotlib thingy using google

Instructions
 1. Process the Text
 - Convert to Lowercase
 - Remove Punctuation & Numbers
 - Tokenization (Splitting text into individual words or tokens)
 - Removing Stop Words (e.g. "and", "the", "of")
 - Stemming/Lemmatization (reducing words to their base/root form) *optional

 2. Vectorize Text
 - Convert text into numerical format using TF-IDF or embeddings
 - TF-IDF weights the words based on their frequency in a document against their frequency across all documents
 - Word embeddings using Word2Vec, CloVe, or FastText can be used to convert words into vectors (better)

 3. Dimensionality Reduction with t-SNE
 - Choosing parameters (starting point 5 - 50, learning rate 10-1000)
 - Scaling - Might need to scae feature vectors before applying (MinMAxScaler or StandardScaler)
 - Applying t-SNE to reduce dimensionality of vectorized hymns (from scikit-learn)

 4. Visualization
 - Plotting (Matplotlib) to create a scatter plot of 2D points
 - Enhancements to make visualization more informative (optional?)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
# Import hymns
hymns = []

def get_hymns(): 
    h = input("Enter the hymn number (1-525). Press enter if finished. ")
    if h:
        hymns.append(int(h))
        get_hymns()
    else:
        return hymns
    
get_hymns()

In [ ]:
# 1. Process the Text


In [ ]:
# extra example stuff
from sklearn.datasets import load_digits

digits = load_digits(n_class=6) # they have a built in dataset of numbers from 0 to n_class
X, y = digits.data, digits.target # X is the data (image i believe), y is the correct value/label... I think
n_samples, n_features = X.shape
n_neighbors = 30 # inspect neighbourhood of nearest 30 points

import matplotlib.pyplot as plt

# print out examples of data
fig, axs = plt.subplots(nrows=10, ncols=10, figsize=(6, 6))
for idx, ax in enumerate(axs.ravel()):
    ax.imshow(X[idx].reshape((8, 8)), cmap=plt.cm.binary)
    ax.axis("off")
_ = fig.suptitle("A selection from the 64-dimensional digits dataset", fontsize=16)

In [ ]:
# this is a helper function to plot embeddings in a 2D space

import numpy as np
from matplotlib import offsetbox

from sklearn.preprocessing import MinMaxScaler


def plot_embedding(X, title):
    _, ax = plt.subplots()
    X = MinMaxScaler().fit_transform(X)

    for digit in digits.target_names:
        ax.scatter(
            *X[y == digit].T,
            marker=f"${digit}$",
            s=60,
            color=plt.cm.Dark2(digit),
            alpha=0.425,
            zorder=2,
        )
    shown_images = np.array([[1.0, 1.0]])  # just something big
    for i in range(X.shape[0]):
        # plot every digit on the embedding
        # show an annotation box for a group of digits
        dist = np.sum((X[i] - shown_images) ** 2, 1)
        if np.min(dist) < 4e-3:
            # don't show points that are too close
            continue
        shown_images = np.concatenate([shown_images, [X[i]]], axis=0)
        imagebox = offsetbox.AnnotationBbox(
            offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r), X[i]
        )
        imagebox.set(zorder=1)
        ax.add_artist(imagebox)

    ax.set_title(title)
    ax.axis("off")

In [ ]:
# this outlines a BUNCH of diff embedding space representations, TSNE is one of them, and you'll later see why it's the best one

from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.manifold import (
    MDS,
    TSNE,
    Isomap,
    LocallyLinearEmbedding,
    SpectralEmbedding,
)
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.random_projection import SparseRandomProjection

embeddings = {
    "Random projection embedding": SparseRandomProjection(
        n_components=2, random_state=42
    ),
    "Truncated SVD embedding": TruncatedSVD(n_components=2),
    "Linear Discriminant Analysis embedding": LinearDiscriminantAnalysis(
        n_components=2
    ),
    "Isomap embedding": Isomap(n_neighbors=n_neighbors, n_components=2),
    "Standard LLE embedding": LocallyLinearEmbedding(
        n_neighbors=n_neighbors, n_components=2, method="standard"
    ),
    "Modified LLE embedding": LocallyLinearEmbedding(
        n_neighbors=n_neighbors, n_components=2, method="modified"
    ),
    "Hessian LLE embedding": LocallyLinearEmbedding(
        n_neighbors=n_neighbors, n_components=2, method="hessian"
    ),
    "LTSA LLE embedding": LocallyLinearEmbedding(
        n_neighbors=n_neighbors, n_components=2, method="ltsa"
    ),
    "MDS embedding": MDS(n_components=2, n_init=1, max_iter=120, n_jobs=2),
    "Random Trees embedding": make_pipeline(
        RandomTreesEmbedding(n_estimators=200, max_depth=5, random_state=0),
        TruncatedSVD(n_components=2),
    ),
    "Spectral embedding": SpectralEmbedding(
        n_components=2, random_state=0, eigen_solver="arpack"
    ),
    "t-SNE embedding": TSNE(
        n_components=2,
        n_iter=500,
        n_iter_without_progress=150,
        n_jobs=2,
        random_state=0,
    ),
    "NCA embedding": NeighborhoodComponentsAnalysis(
        n_components=2, init="pca", random_state=0
    ),
}

In [ ]:
# run it

from time import time

projections, timing = {}, {}
for name, transformer in embeddings.items():
    if name.startswith("Linear Discriminant Analysis"):
        data = X.copy()
        data.flat[:: X.shape[1] + 1] += 0.01  # Make X invertible
    else:
        data = X

    print(f"Computing {name}...")
    start_time = time()
    projections[name] = transformer.fit_transform(data, y)
    timing[name] = time() - start_time

for name in timing:
    title = f"{name} (time {timing[name]:.3f}s)"
    plot_embedding(projections[name], title)

plt.show()

# Hymn lyric section

In [ ]:
import lyricsgenius
# you will want to run `pip install lyricsgenius` in your terminal,
# again while you see (choir_network) so it means you are adding a library to this environment.
# if pip doesn't work, you need to install pip to conda

file_path = '/Users/markchen/Desktop/genius.txt' # replace this with wherever the file I gave you is
token = "x6BMSqzUa..." # it is NEVER good to show api tokens in code.
# In a workplace, it means security issues, and you'd get in trouble.

# read the last line from the important file to get the token, without exposing it to github/public
try:
    with open(file_path, 'r') as file:
        for i, line in enumerate(file, start=1):
            if i == 6:
                token = line.strip() # set the token as what is in the file, which will be used for the Genius API for lyrics
                break
except FileNotFoundError:
    print(f"The file at {file_path} was not found.")

# function to get lyrics
def get_hymn_lyrics(title):
    genius = lyricsgenius.Genius(token)
    song = genius.search_song(title)
    if song:
        print(song.lyrics)
    else:
        print(f"No lyrics found for '{title}'.")


In [ ]:
hymn_title = "Amazing Grace"
get_hymn_lyrics(hymn_title)
# now let's see if we can just keep things simple, and clean/parse just for verse 1 of every hymn in titles.txt
# Save it in a CSV i guess?
# number,title,verse1,failed_or_not
